## Instituto Federal da Paraíba
## Disciplina: Mineração de Dados
## Professor: Paulo Ribeiro
## Alunos: Filipe Salomão, Hércules de Sousa e Matheus Alves
##
## Projeto: Classificação de Gêneros Musicais

Importanto bibliotecas necessárias:

In [ ]:
import librosa
import numpy as np
import os
import pandas as pd
from joblib import load
from scipy.stats import kurtosis
from scipy.stats import skew
import warnings

Reutilizando a função **_get_features_**:

In [ ]:
def get_features(y, sr, n_fft=1024, hop_length=512):
    features = {}

    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_rolloff'] = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zero_crossing_rate'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rms'] = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['onset_strength'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['spectral_contrast'] = librosa.feature.spectral_contrast(y=y, sr=sr).ravel()
    features['spectral_bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_flatness'] = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length).ravel()

    mfcc = librosa.feature.mfcc(y=y, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)

    for index, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(index)] = v_mfcc.ravel()

    def get_moments(descriptors):
        result = {}

        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)

        return result

    additional_features = get_moments(features)
    additional_features['tempo'] = librosa.feature.tempo(y=y, sr=sr)[0]

    return additional_features

Diretório onde estão as músicas a serem testadas:

In [ ]:
music_directory = 'music'

Obtendo os gêneros presentes no diretório:

In [ ]:
music_genres = os.listdir(music_directory)

Carregando todos os modelos para testá-los:

In [ ]:
pipe_lr = load('./models/pipe_lr.joblib')

In [ ]:
pipe_dt = load('./models/pipe_dt.joblib')

In [ ]:
pipe_rf = load('./models/pipe_rf.joblib')

In [ ]:
pipe_svm = load('./models/pipe_svm.joblib')

In [ ]:
pipe_knn = load('./models/pipe_knn.joblib')

In [ ]:
pipe_nb = load('./models/pipe_nb.joblib')

A função **_test_model_** recebe um pipeline de modelo, um diretório contendo músicas e uma lista de gêneros musicais. Ela realiza predições das músicas de cada gênero utilizando o modelo fornecido e imprime os resultados das predições. Durante a execução, são ignorados os avisos gerados.

In [ ]:
def test_model(pipe, directory, genres):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        results = []

        for genre in genres:
            print(f'Realizando predições de músicas de {genre}...')

            genre_directory = os.path.join(directory, genre)
            genre_music_files = os.listdir(genre_directory)

            for music in genre_music_files:
                signal, sr = librosa.load(os.path.join(genre_directory, music))
                features = get_features(signal, sr)
                song = pd.DataFrame([features])

                pred = pipe.predict(song)[0]

                results.append((genre, music, pred))

        for (genre, music, pred) in results:
            print(f'Gênero {genre}: {music} foi predito como música de {pred}')